<a href="https://colab.research.google.com/github/gourab-sinha/Machine_Learning/blob/master/Decision%20Tree/Project_Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn import datasets
import pandas as pd
import numpy as np
import math

In [0]:

class DecisionTree:
  classes = ['setosa', 'versicolor', 'virginica']
  
  
  # Information gain
  def __information_gain(self,X,Y,targets,feature):
    X_new = X[feature].values
    X_new.sort()
    # print(X_new,len(X_new))
    total = Y.shape[0]
    total_setosa =  Y[Y[targets]==0].shape[0]
    total_versicolor = Y[Y[targets]==1].shape[0]
    total_virginica = Y[Y[targets]==2].shape[0]
    level_entropy = -((total_setosa/total)*math.log2(total_setosa/total) + 
                      (total_versicolor/total)*math.log2(total_versicolor/total) + 
                      (total_versicolor/total)*math.log2(total_versicolor/total))
    
    max_info_gain = 0
    level_threshold = 0
    max_gain_ratio = 0
    for i in range(1,len(X_new)):
      # Define threshold 
      threshold = (X_new[i-1]+X_new[i])/2
      # print(threshold)
      # Data points below or equal to threshold
      X_below_threshold = X[X[feature]<threshold]

      # Data points above threshold 
      X_above_threshold = X[X[feature]>=threshold]


      # Data points below and above
      total_below = X_below_threshold.shape[0]
      total_above = X_above_threshold.shape[0]
      # print(total_below,total_above)

      info_required_below = 0
      if total_below>0:
        # Indices of target values for below threshold
        Y_index_below_threshold = X[X[feature]<threshold].index

        # Targets below threshold
        Y_below = Y.iloc[Y_index_below_threshold]

        # Below or equal threshold
        total_setosa_below = Y_below[Y_below[targets]==0].shape[0]
        total_versicolor_below = Y_below[Y_below[targets]==1].shape[0]
        total_virginica_below = Y_below[Y_below[targets]==2].shape[0]
        # print(total_setosa_below,total_versicolor_below,total_virginica_below)

        info_required_below = 0
        # Information required below threshold
        if total_setosa_below>0:
          info_required_below += (total_setosa_below/total_below)*math.log2(total_setosa_below/total_below)
        if total_versicolor_below>0:
          info_required_below += (total_versicolor_below/total_below)*math.log2(total_versicolor_below/total_below)
        if total_virginica_below>0:
          info_required_below += (total_virginica_below/total_below)*math.log2(total_virginica_below/total_below)
        
        info_required_below = -info_required_below
      
      info_required_above = 0
      if total_above>0:
        # Indices of target values for above threshold
        Y_index_above_threshold = X[X[feature]>=threshold].index

        # Targets above threshold
        Y_above = Y.iloc[Y_index_above_threshold]

        # Above Threshold
        total_setosa_above = Y_above[Y_above[targets]==0].shape[0]
        total_versicolor_above = Y_above[Y_above[targets]==1].shape[0]
        total_virginica_above = Y_above[Y_above[targets]==2].shape[0]

        info_required_above = 0
        # Information required above threshold
        if total_setosa_above>0:
          info_required_above += (total_setosa_above/total_above)*math.log2(total_setosa_above/total_above)
        if total_versicolor_above>0:
          info_required_above += (total_versicolor_above/total_above)*math.log2(total_versicolor_above/total_above)
        if total_virginica_above>0:
          info_required_above += (total_virginica_above/total_above)*math.log2(total_virginica_above/total_above)
        
        info_required_above = -info_required_above

      
      
      # Information required
      info_require = (total_below/total)*info_required_below + (total_above/total)*info_required_above

      # Current gain
      current_info_gain = level_entropy - info_require

      # Split Information
      current_split_info = 0 
      if total_below>0:
        current_split_info += (total_below/total)*math.log2(total_below/total)
      if total_above>0:
        current_split_info += (total_above/total)*math.log2(total_above/total)
      
      current_split_info = -current_split_info

      # Current Gain Ratio 
      current_gain_ratio = 0
      if current_split_info>0:
        current_gain_ratio = current_info_gain/current_split_info
      # Check with previous Max Gain Ratio
      if max_gain_ratio <= current_gain_ratio :
        level_threshold = threshold
        max_info_gain  = current_info_gain
        max_gain_ratio = current_gain_ratio

    
    return max_info_gain, max_gain_ratio, level_entropy, level_threshold




  # Decision Tree 
  def decision_tree(self,X,Y, features,level,targets):  
    types = dict(Y[targets].value_counts().items())
    # print(len(types),len(features))
    if len(features)==0 or len(types)==1:
      print("Level ",level)
      for cat,val in types.items():
        print("Count of ",cat,val)
      print("Current Entropy is ", 0)
      print("Reached leaf Node")
      return 
    
    selected_feature = ""
    max_info_gain = 0
    level_gain_ratio = 0
    level_entropy = 0
    level_threshold = -1 # for continuous
    for feature in features:
      gain, gain_ratio, entropy, threshold = self.__information_gain(X,Y,targets,feature)
      if gain_ratio>=level_gain_ratio:
        selected_feature = feature
        max_gain = gain
        level_gain_ratio = gain_ratio 
        level_entropy = entropy
        level_threshold = threshold  

    # print(level_threshold)
    # return
    count_with_classes = dict(Y[targets].value_counts().items())
    print("Level ", level)
    for key,val in count_with_classes.items():
      print("Count of "+self.classes[key],val)
    print("Current Entropy is ",level_entropy)
    print("Splitting on feature "+str(selected_feature)+ " with gain ratio",level_gain_ratio)
    features.remove(selected_feature)


    X1 = X[X[selected_feature]<=level_threshold].copy()
    X1.index = np.arange(1, len(X1) + 1)
    X2 = X[X[selected_feature]>level_threshold].copy()
    X2.index = np.arange(1, len(X2) + 1)
    Y1_index = X[X[selected_feature]<=level_threshold].index
    Y2_index = X[X[selected_feature]>level_threshold].index
    Y1 = Y.iloc[Y1_index].copy()
    Y1.index = np.arange(1, len(Y1) + 1)
    Y2 = Y.iloc[Y2_index].copy()
    Y2.index = np.arange(1, len(Y2) + 1)
    self.decision_tree(X1,Y1,features,level+1,targets)
    self.decision_tree(X2,Y2,features,level+1,targets)
    
    # else:
    #   different_val_on_selected_feature = set(X[selected_feature].values)
    #   for val in different_val_on_selected_feature:
    #     X_new = X[X[selected_feature]==val]
    #     Y_new_index = X[X[selected_feature]==val].index
    #     Y_new = Y.iloc[Y_new_index]
    #     self.decision_tree(X_new,Y_new,features,level+1,targets)

# """
# Level  0
# Count of  0(False)  =  1
# Count of  1(True)  =  3
# Current Entropy  is =  0.811278124459
# Splitting on feature  X1  with gain ratio  0.311278124459
# """

In [143]:
iris = datasets.load_iris()
X = pd.DataFrame(iris.data,columns = iris.feature_names)
Y = pd.DataFrame(iris.target,columns = ['target'])
features = list(iris.feature_names)
print(features)

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [0]:
dc_tree = DecisionTree()
dc_tree.decision_tree(X,Y,features,0,"target")

In [24]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [109]:
math.log2(0/1)

ValueError: ignored